In [2]:
# import libraries
import pandas as pd
import scipy.stats
import statsmodels.stats.multitest
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# project specific libs
import os
import matplotlib.pyplot as plt

In [3]:
# set (local) path
path = '/Users/KevinBu/Desktop/clemente_lab/Projects/ampaim/'

In [4]:
###
# Load AB mapping file
###

# get AMPAIM+EISER mapping file w metadata; drop row 1
df_map_AB = pd.read_csv(path + 'inputs/adamcantor22_Cross_Disease_Pilot_0/Qiime2_0/qiime_mapping_file.tsv', sep='\t', header=0, index_col=0)

# get first row for any future operations
first_row = df_map_AB.reset_index().iloc[0,]
first_row_df = pd.DataFrame(first_row).T
df_map_AB = df_map_AB.iloc[1:,]


###
# Specific replacements
###

# 526-0-twin-psaplate308 is missing a dash
df_map_AB = df_map_AB.rename(index={'526-0-twin-psaplate308': '526-0-twin-psa-plate308'})

###
# General replacements
###

# refrain from dropping NA's because you might need the columns for Q2 and you need to smoothly concat the first row

# convert index to str from float etc.
df_map_AB.index = df_map_AB.index.map(str)

# switch EISER diagnosis to eiser from NA
df_map_AB.loc[df_map_AB['Project'] == 'eiser', 'Diagnosis'] = 'eiser'

# RBB mapping file doesn't have info on duplicate controls, and the glass control info is extracted only from a subset of the names
# so we need to grab that info from AB's mapping file
samp_to_ctrlstatus = df_map_AB['Project'].to_dict()

# grab AB non-microteach samples
df_map_AB_filt = df_map_AB[df_map_AB['Project'].isin(['eiser','TWIN_PSA','glass_control','neg_control','duplicate_control'])]
df_map_AB_filt.head()

,BarcodeSequence,LinkerPrimerSequence,AmpliconWell,BSA,BSASeverityByBSA,CCPtiter,CRP,CurrentBiologics,CurrentIntralesionalSteroids,CurrentMTX,...,StudyType,SubjectType,BirthYear,HostSubjectId,Nationality,Sex,SpecimenType,UberonCodeType,Weight,WeightDateCollected
#SampleID,,,,,,,,,,,,,,,,,,,,,
116783-plate305,CCTCGCATGACC,CCGGACTACHVGGGTWTCTAAT,A1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Human,NaN,116783,NaN,NaN,fecal,NaN,NaN,NaN
116784-plate305,CGCGCAAGTATT,CCGGACTACHVGGGTWTCTAAT,B1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Human,NaN,116784,NaN,NaN,fecal,NaN,NaN,NaN
116785-plate305,AAGGCGCTCCTT,CCGGACTACHVGGGTWTCTAAT,C1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Human,NaN,116785,NaN,NaN,fecal,NaN,NaN,NaN
116786-plate305,CGCAATGAGGGA,CCGGACTACHVGGGTWTCTAAT,D1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Human,NaN,116786,NaN,NaN,fecal,NaN,NaN,NaN
116787-plate305,ACGGCGTTATGT,CCGGACTACHVGGGTWTCTAAT,E1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Human,NaN,116787,NaN,NaN,fecal,NaN,NaN,NaN


In [5]:
###
# Prepare for merge with RB mapping file
###

# load RB mapping file
df_map_RB = pd.read_csv(path + 'inputs/qiime_mapping_file_final_062123_RBB_ESR_CRPmod.csv', index_col=0)

# skip row with q2types; 140 columns
df_map_RB = df_map_RB.iloc[1:,]

# exclude unaffected since we are using TWINS_PSA from AB and controls from AB
# we only want microteach samples
df_map_RB = df_map_RB[df_map_RB['Diagnosis'].isin(['unaffected','healthy','ss','sle','cd','RA','psa','pso'])]

# drop duplicate samples that appear in AB and that are controls per AB mapping file
for s in list(df_map_RB.index.values):
    if samp_to_ctrlstatus[s] in ['glass_control','duplicate_control','neg_control']:
        df_map_RB = df_map_RB.drop(s)
        
        
# merge with AB
df_meta = pd.concat([df_map_AB_filt, df_map_RB], axis=0)

# drop 'Separate' and 'Together'; presumably Q2 artifacts
df_meta = df_meta.drop(['Separate','Together'], axis=1)

# add in Project col from AB that describes controls
df_meta['Project'] = df_meta.index.map(samp_to_ctrlstatus)

###
# Specific replacements
###

# replace HxOtherDMARDs
df_meta.loc['540-0-twin-psa-plate308','HxOtherDMARDs'] = 'N'

### 
# General replacements
###

# specify float cols for later
float_cols = ['Age','BSA','CRP','DAS28','ESR','PhysicianGlobalPsA','RAPID3','SJC','TJC']

# Replace Y,N with 1,0 
df_meta = df_meta.replace({'Y':1,'N':0})

# create dict mapping diagnosis to samples
diag_to_samp = {}
for d in list(set(df_meta['Diagnosis'].values)):
    diag_to_samp[d] = list(df_meta[df_meta['Diagnosis'] == d].index.values)

### 
# Create new mapping files
### 

def export_q2(df, first_row_df):
    df_q2 = df.reset_index()
    df_q2 = pd.concat([first_row_df, df_q2])
    df_q2 = df_q2.set_index('#SampleID')
    df_q2.loc['#q2:types',:] = 'categorical'
    df_q2 = df_q2.reset_index()
    return df_q2

# (1) filter out controls
df_meta = df_meta[df_meta['Project'].isin(['eiser','microteach','TWIN_PSA'])]

# (2) filter out treatments
# drop TWIN PSA and PSO samples that have treatment on biologics, DMARDS, MTX
drop_samples = ['275-psa-plate307', '475-psa-plate307', '542-0-twin-psa-plate308']
df_meta = df_meta.drop(drop_samples)

# remove samples that are treated other than above
drops = []
for v in ['CurrentBiologics', 'CurrentOtherDMARDs', 'CurrentMTX', 'HxOtherDMARDs']:
    df = df_meta[df_meta[v].isin(['Y',1])]
    drops = drops + list(df.index.values)

df_meta = df_meta.drop(drops)
print('Dropped this many samples: ' + str(len(drops)))

# drop eiser samples from downstream analysis
df_meta = df_meta[df_meta['Diagnosis'] != 'eiser']

# drop CD 
df_meta = df_meta[df_meta['Diagnosis'] != 'cd']

# export to Q2
df_meta_q2 = export_q2(df_meta, first_row_df)

# subset on twins_psa only
df_meta = df_meta[df_meta['Project'] == 'TWIN_PSA']

df_meta['Diagnosis'].value_counts()

Dropped this many samples: 18


/var/folders/48/kjsfkfqd4z1f5svhyyb5kvdw0000gn/T/ipykernel_65548/1209467571.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'categorical' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_q2.loc['#q2:types',:] = 'categorical'


Diagnosis
unaffected    25
pso           10
psa            1
Name: count, dtype: int64

In [16]:
# exploratory analysis
df = df_meta.dropna(axis=1,how='all')

# drop useless var
drop_col = ['BarcodeSequence', 'LinkerPrimerSequence', 'AmpliconWell', 'BSA',
      'NumberSamplePlate', 'Plate', 'SkinSwabCollected',
       'StoolCollectDate', 'StoolCollected', 'StoolReceivedDate',
       'TemplateDNAwell','ContactEmail', 'ContactName',
       'PrimaryInvestigator', 'AmpWellPlate', 
       'RawDataNotes', 'RawDataProtocolID', 'ResultsProtocolID',
       'MmedsSampleID', 'SampleProtocolID', 'SpecimenID', 'StudyName',
       'SubjectType', 'HostSubjectId',  'SpecimenType', 'OtherSubjectID']
df = df.drop(drop_col,axis=1)

# drop na; this was 0
df = df.dropna(how='all',axis=1)

# remove uniform columns
for i, v in enumerate(list(df.columns.values)):
    if len(df[v].value_counts()) == 1:
        df = df.drop(v, axis=1)

# how are they paired
df.head()

,CurrentTopicalSteroids,Dactylitis,Diagnosis,DiseaseDuration,DrugNaive,Enthesitis,HxBiologics,ProbioticsInPast3Months,PsOtype,SJC,TJC,Age,AbxInPast3Months,Sex
#SampleID,,,,,,,,,,,,,,
518-0-twin-psa-plate308,0.0,0,pso,21.0,1.0,1.0,0.0,1.0,NaN,0.0,2.0,43.0,0.0,F
519-0-twin-psa-plate308,NaN,0,unaffected,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,43.0,NaN,F
521-0-twin-psa-plate308,NaN,0,unaffected,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,34.0,NaN,M
524-0-twin-psa-plate308,1.0,0,pso,35.0,1.0,0.0,0.0,0.0,"Plaque, scalp, nail",0.0,0.0,54.0,0.0,M
525-0-twin-psa-plate308,0.0,0,pso,5.0,1.0,0.0,0.0,0.0,Nail,0.0,1.0,54.0,0.0,M
